<a href="https://colab.research.google.com/github/darp-cs/Computer-Architecture/blob/main/Projectv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bi#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import math
import struct
import random
import sys


def rawDG(values):
    vals = values
    for key in vals:
        if key + 1 < len(vals):
            if vals[key].destination in vals[key + 1].depends:
                vals[key + 1].depends(vals[key].destination)
                vals[key + 1].depends.remove(vals[key].destination)
                vals[key].no_op(key, 3)
                return None
        if key + 2 < len(vals):
            if vals[key].destination in vals[key + 2].depends:
                vals[key + 2].depends(vals[key].destination)
                vals[key + 2].depends.remove(vals[key].destination)
                vals[key].no_op(key, 2)
                return None
        if key + 3 < len(vals):
            if vals[key].destination in vals[key + 3].depends:
                vals[key + 3].depends(vals[key].destination)
                vals[key + 3].depends.remove(vals[key].destination)
                vals[key].no_op(key, 1)
                return None

class Instruction:

    __inst_type = ""
    __first_reg = ""
    __second_reg = ""
    __destination = 0
    __to_decode = ""
    __depends_on = []
    __non_binary = 0.0
    __machine_code = ""
    __binary_float = ""
    __switched_depends = False
    __opCode = (
        "000000", "000001", "000010", "000011", "000100", "000101", "000110", "000111", "001000", "001001", "001010",
        "001011", "001100", "001101", "001110", "001111", "010000", "010001", "010010", "010011", "010100", "010101",
        "010110")
    __regNum = ("0000", "0001", "0010", "0011", "0100", "0101", "0110", "0111", "1000", "1001", "1010", "1011", "1100",
                "1101", "1110", "1111")
    __memoryLocation = 0
    __no_ops = {}
    __reg = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    __memory = {}

    def __init__(self, t=""):
        self.__to_decode = t

    @property
    def getCoded(self):
        return self.__to_decode

    @property
    def inst_type(self):
        return self.__inst_type

    @inst_type.setter
    def inst_type(self, value):
        self.__inst_type = value

    @property
    def first_reg(self):
        return self.__first_reg

    @first_reg.setter
    def first_reg(self, value):
        self.__first_reg = value

    @property
    def second_reg(self):
        return self.__second_reg

    @second_reg.setter
    def second_reg(self, value):
        self.__second_reg = value

    @property
    def destination(self):
        return self.__destination

    @destination.setter
    def destination(self, value):
        self.__destination = value

    @property
    def depends(self):
        return self.__depends_on

    @depends.setter
    def depends(self, value):
        self.__depends_on = value

    @property
    def type(self):
        return self.__inst_type

    @type.setter
    def setType(self, value):
        self.__inst_type = value

    @property
    def machine_code(self):
        return self.__machine_code

    @property
    def switched_depends(self):
        return self.__switched_depends

    @switched_depends.setter
    def switched_depends(self, value):
        self.__switched_depends = value

    @property
    def no_ops(self):
        return self.__no_ops

    @no_ops.setter
    def no_ops(self, key, value):
        self.__no_ops[key] = value

    @property
    def reg(self):
        return self.__reg

    def update_reg(self, d):
        for x in range(len(d)):
            self.__reg[x] = d[x]

    @property
    def memory(self):
        return self.__memory

    def update_memory(self, d):
        for key in d:
            self.__memory[key] = d[key]

    def decode(self):
        codes = self.__to_decode.split()

        if codes[0] == "SET":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[0]
            return self.binarySet(codes)

        if codes[0] == "LOAD":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[1]
            return self.binaryLoad(codes)

        if codes[0] == "STORE":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[2]
            return self.binaryStore(codes)

        if codes[0] == "MOVE":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[3]
            return self.binaryMove(codes)

        if codes[0] == "FADD":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[4]
            return self.binaryAdd(codes)

        if codes[0] == "FSUB":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[5]
            return self.binarySubtract(codes)

        if codes[0] == "FNEG":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[6]
            return self.binaryNegate(codes)

        if codes[0] == "FMUL":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[7]
            return self.binaryMultiply(codes)

        if codes[0] == "FDIV":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[8]
            return self.binaryDivide(codes)

        if codes[0] == "FLOOR":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[9]
            return self.binaryFloor(codes)

        if codes[0] == "CEIL":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[10]
            return self.binaryCeiling(codes)

        if codes[0] == "ROUND":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[11]
            return self.binaryRound(codes)

        if codes[0] == "FABS":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[12]
            return self.binaryAbsValue(codes)

        if codes[0] == "MIN":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[13]
            return self.binaryMinimum(codes)

        if codes[0] == "MAX":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[14]
            return self.binaryMaximum(codes)

        if codes[0] == "POW":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[15]
            return self.binaryPower(codes)

        if codes[0] == "EXP":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[16]
            return self.binaryExponent(codes)

        if codes[0] == "SQRT":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[17]
            return self.binarySquareRoot(codes)

        if codes[0] == "B":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[18]
            return self.binaryBranch(codes)

        if codes[0] == "BZ":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[19]
            return self.binaryBranchZ(codes)

        if codes[0] == "BN":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[20]
            return self.binaryBranchN(codes)

        if codes[0] == "NOP":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[21]
            return self.binaryNoOp(codes)

        if codes[0] == "HALT":
            self.__inst_type = codes[0]
            self.__machine_code += self.__opCode[22]
            return self.binaryHalt(codes)
        else:
            if "<" in codes[0] or ">" in codes[0]:
                temp = codes[0].replace("<", '')
                temp = codes[0].replace(">", ' ')
                temp = temp.split()
                loc = temp[0]
                loc = loc[1:(len(loc))]
                val = temp[1]
                val = val[1:(len(val))]
                self.__destination = int(loc)
                value = float(val)
                return self.memory_write(value)

    def execute(self):
        if self.__inst_type == "SET":
            return self.write(self.__non_binary)
        if self.__inst_type == "LOAD":
            self.__memoryLocation = int(self.__reg[int(self.__memoryLocation)])
            return self.write(self.__memory[self.__memoryLocation])
        if self.__inst_type == "STORE":
            registerNum1 = int(self.__first_reg)
            product = self.__reg[registerNum1]
            return self.memory_write(product)
        if self.__inst_type == "MOVE":
            registerNum1 = int(self.__first_reg)
            product = self.__reg[registerNum1]
            return self.write(product)
        if self.__inst_type == "FADD":
            registerNum1 = int(self.__first_reg)
            registerNum2 = int(self.__second_reg)
            product = self.__reg[registerNum1] + self.__reg[registerNum2]
            return self.write(product)
        if self.__inst_type == "FSUB":
            registerNum1 = int(self.__first_reg)
            registerNum2 = int(self.__second_reg)
            product = self.__reg[registerNum1] - self.__reg[registerNum2]
            return self.write(product)
        if self.__inst_type == "FNEG":
            registerNum1 = int(self.__first_reg)
            product = self.__reg[registerNum1] * -1
            return self.write(product)
        if self.__inst_type == "FMUL":
            registerNum1 = int(self.__first_reg)
            registerNum2 = int(self.__second_reg)
            product = self.__reg[registerNum1] * self.__reg[registerNum2]
            return self.write(product)
        if self.__inst_type == "FDIV":
            registerNum1 = int(self.__first_reg)
            registerNum2 = int(self.__second_reg)
            product = self.__reg[registerNum1] / self.__reg[registerNum2]
            return self.write(product)
        if self.__inst_type == "ROUND":
            registerNum1 = int(self.__first_reg)
            registerNum2 = int(self.__second_reg)
            product = self.__reg[registerNum1] + self.__reg[registerNum2]
            return self.write(product)
        if self.__inst_type == "FABS":
            registerNum1 = int(self.__first_reg)
            product = abs(self.__reg[registerNum1])
            return self.write(product)
        if self.__inst_type == "MIN":
            registerNum1 = int(self.__first_reg)
            registerNum2 = int(self.__second_reg)
            product = min(self.__reg[registerNum1], self.__reg[registerNum2])
            return self.write(product)
        if self.__inst_type == "MAX":
            registerNum1 = int(self.__first_reg)
            registerNum2 = int(self.__second_reg)
            product = max(self.__reg[registerNum1] + self.__reg[registerNum2])
            return self.write(product)
        if self.__inst_type == "POW":
            registerNum1 = int(self.__first_reg)
            registerNum2 = int(self.__second_reg)
            product = pow(self.__reg[registerNum1],self.__reg[registerNum2])
            return self.write(product)
        if self.__inst_type == "EXP":
            registerNum1 = int(self.__first_reg)
            registerNum2 = int(self.__second_reg)
            product = exp(self.__reg[registerNum1], self.__reg[registerNum2])
            return self.write(product)
        if self.__inst_type == "SQRT":
            registerNum1 = int(self.__first_reg)
            product = sqrt(self.__reg[registerNum1])
            return self.write(product)
        if self.__inst_type == "B":
            return
        if self.__inst_type == "BZ":
            return
        if self.__inst_type == "BN":
            return
        if self.__inst_type == "NOP":
            return
        if self.__inst_type == "HALT":
            return

    def write(self, value):
        self.__reg[self.__destination] = value
        #add depend
    def memory_write(self, value):
        self.__memory[self.destination] = value
        #add depend
    def binarySet(self, codes):
        self.__destination = codes[1]
        temp = codes[2]
        self.__destination = self.__destination[1:(len(self.__destination) - 1)]
        temp = temp[1:(len(temp))]
        self.__machine_code = (self.__opCode[0] + self.__regNum[int(self.__destination)])
        self.__non_binary = float(temp)
        self.__binary_float, size = self.__dec_to_bin(self.__non_binary)
        size = len(self.__binary_float)
        self.__machine_code = self.__fill_middle(self.__machine_code, size)
        self.__machine_code = self.__machine_code + str(self.__binary_float)
        self.__destination = int(self.__destination)
        #add depend
        return None

    def binaryLoad(self, codes):
        self.__destination = codes[1]
        temp = codes[2]
        self.__destination = self.__destination[1:(len(self.__destination) - 1)]
        temp = temp[1:(len(temp))]
        self.__memoryLocation = temp[1:(len(temp))]
        self.__machine_code = (self.__opCode[1] + self.__regNum[int(self.__destination)] + self.__regNum[int(temp)])
        self.__memoryLocation = int(self.__reg[int(temp)])
        self.__machine_code = self.__fill(self.__machine_code)
        self.__destination = int(self.__destination)
        #add depend
        return None

    def binaryStore(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        self.__memoryLocation = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn))]
        final = (self.__opCode[2] + self.__regNum[int(self.__memoryLocation)] + self.__regNum[int(self.__first_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__destination = int(self.__destination)
        #add depend
        return None

    def binaryMove(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        self.__destination = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn))]
        final = (self.__opCode[3] + self.__regNum[int(self.__destination)] + self.__regNum[int(self.__first_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__destination = int(self.__destination)
        #add depend
        return None

    def binaryAdd(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        thirdIn = codes[3]
        self.__destination = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn) - 1)]
        self.__second_reg = thirdIn[1:len(thirdIn)]
        final = (self.__opCode[4] + self.__regNum[int(self.__destination)] + self.__regNum[int(self.__first_reg)] +self.__regNum[int(self.__second_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__depends_on.append(self.__second_reg)
        self.__destination = int(self.__destination)
        return None

    def binarySubtract(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        thirdIn = codes[3]
        self.__destination = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn) - 1)]
        self.__second_reg = thirdIn[1:len(thirdIn)]
        final = (self.__opCode[5] + self.__regNum[int(self.__destination)] + self.__regNum[int(self.__first_reg)] + self.__regNum[int(self.__second_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__depends_on.append(self.__second_reg)
        self.__destination = int(self.__destination)
        return None

    def binaryNegate(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        self.__destination = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn))]
        final = (self.__opCode[6] + self.__regNum[int(self.__destination)] + self.__regNum[int(self.__first_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__destination = int(self.__destination)
        return None

    def binaryMultiply(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        thirdIn = codes[3]
        self.__destination = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn) - 1)]
        self.__second_reg = thirdIn[1:len(thirdIn)]
        final = (self.__opCode[7] + self.__regNum[int(self.__destination)] + self.__regNum[int(self.__first_reg)] + self.__regNum[int(self.__second_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__depends_on.append(self.__second_reg)
        self.__destination = int(self.__destination)
        return None

    def binaryDivide(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        thirdIn = codes[3]
        self.__destination = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn) - 1)]
        self.__second_reg = thirdIn[1:len(thirdIn)]
        final = (self.__opCode[8] + self.__regNum[int(self.__destination)] + self.__regNum[int(self.__first_reg)] + self.__regNum[int(self.__second_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__depends_on.append(self.__second_reg)
        self.__destination = int(self.__destination)
        return None

    def binaryFloor(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        self.__destination = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn))]
        final = (self.__opCode[9] + self.__regNum[int(self.destination)] + self.__regNum[int(self.__first_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__destination = int(self.__destination)
        return None

    def binaryCeiling(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        self.__destination = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn))]
        final = (self.__opCode[10] + self.__regNum[int(self.__destination)] + self.__regNum[int(self.__first_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__destination = int(self.__destination)
        return None

    def binaryRound(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        self.__destination = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn))]
        final = (self.__opCode[11] + self.__regNum[int(self.__destination)] + self.__regNum[int(self.__first_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__destination = int(self.__destination)
        return None

    def binaryAbsValue(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        self.__destination = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn))]
        final = (self.__opCode[12] + self.__regNum[int(self.__destination)] + self.__regNum[int(self.__first_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__destination = int(self.__destination)
        return None

    def binaryMinimum(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        thirdIn = codes[3]
        self.__destination = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn) - 1)]
        self.second_reg = thirdIn[1:len(thirdIn)]
        final = (self.__opCode[13] + self.__regNum[int(self.__destination)] + self.__regNum[int(self.__first_reg)] + self.__regNum[int(self.__second_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__depends_on.append(self.__second_reg)
        self.__destination = int(self.__destination)
        return None

    def binaryMaximum(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        thirdIn = codes[3]
        self.__destination = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn) - 1)]
        self.__second_reg = thirdIn[1:len(thirdIn)]
        final = (self.__opCode[14] + self.__regNum[int(self.__destination)] + self.__regNum[int(self.__first_reg)] + self.__regNum[int(self.__second_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__depends_on.append(self.__second_reg)
        self.__destination = int(self.__destination)
        return None

    def binaryPower(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        thirdIn = codes[3]
        self.__destination = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn) - 1)]
        self.__second_reg = thirdIn[1:len(thirdIn)]
        final = (self.__opCode[15] + self.__regNum[int(self.__destination)] + self.__regNum[int(self.__first_reg)] + self.__regNum[int(self.__second_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__depends_on.append(self.__second_reg)
        self.__destination = int(self.__destination)
        return None

    def binaryExponent(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        self.__destination = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn))]
        final = (self.__opCode[16] + self.__regNum[int(self.__destination)] + self.__regNum[int(self.__first_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__destination = int(self.__destination)
        return None

    def binarySquareRoot(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        self.__destination = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = secIn[1:(len(secIn))]
        final = (self.__opCode[17] + self.__regNum[int(self.__destination)] + self.__regNum[int(self.__first_reg)])
        self.__machine_code = self.__fill(final)
        self.__depends_on.append(self.__first_reg)
        self.__destination = int(self.__destination)
        return None

    def binaryBranch(self, codes):
        firstIn = codes[1]
        firstIn = firstIn[1:(len(firstIn))]
        secIn = self.int_to_bin(codes[2])
        final = (self.__opCode[18] + self.__regNum[(int)(firstIn) + secIn])
        final = self.__fillMiddle(final, 8)
        final += secIn
        return None

    def binaryBranchZ(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        firstIn = firstIn[1:(len(firstIn) - 1)]
        self.__first_reg = int(firstIn)
        secIn = self.int_to_bin(codes[2], 8)
        final = (self.__opCode[19] + self.__regNum[int(firstIn)])
        final = self.__fill_middle(final, 8)
        final += secIn
        self.__machine_code = final
        return None

    def binaryBranchN(self, codes):
        firstIn = codes[1]
        secIn = codes[2]
        firstIn = firstIn[1:(len(firstIn) - 1)]
        secIn = int2bin(codes[2], 8)
        final = (opCode[20] + regNum[(int)(firstIn)])
        final = fillMiddle(final, 8)
        final += secIn
        return None

    def binaryNoOp(self, codes):
        self.__machine_code = self.__fill(self.__opCode[21])

    def binaryHalt(self, codes):
        self.__machine_code = self.__fill(self.__opCode[22])
        return None

    def int_to_bin(self, integer, digits):
        integer = int(integer)
        if integer >= 0:
            return bin(integer)[2:].zfill(digits)
        else:
            return bin(2 ** digits + integer)[2:]

    """
    Obatained from stack overflow to restructure a future part of the code
    Function dec2bin
    """

    def __dec_to_bin(self, f):

        if f >= 1:
            g = int(math.log(f, 2))
        else:
            g = -1
        h = g + 1
        ig = math.pow(2, g)
        st = ""
        while f > 0 or ig >= 1:
            if f < 1:
                if len(st[h:]) >= 10:  # 10 fractional digits max
                    break
            if f >= ig:
                st += "1"
                f -= ig
            else:
                st += "0"
            ig /= 2
        st = st[:h] + st[h:]
        size = len(st)
        return st, size

    def __fill(self, code):
        while (len(code) < 32):
            code += "0"
        return code

    def __fill_middle(self, code, binSize):
        size = 32 - (len(code) + binSize)
        for i in range(size):
            code += "0"
        return code


def intoDict(inFile):
    x = 0
    instructions = {}
    file = open(inFile, 'r')
    for line in file:
        instructions[x] = Instruction(line)
        x += 1
    return instructions


def numToBin(num):
    get_bin = lambda x: format(x, 'b')
    return get_bin(num)


def main():
    fileName = 'testInput.txt'
    registers = []
    memory = {}
    instructions = intoDict(fileName)
    x = 0
    for key in instructions:
        instructions[key].decode()
        memory = instructions[key].memory
    for key in instructions:
        instructions[key].update_memory(memory)
        instructions[key].update_reg(registers)
        instructions[key].execute()
        registers = instructions[key].reg
        memory = instructions[key].memory
        registers = instructions[key].reg
    print(registers)
    print(memory)
    rawDG(instructions)


if __name__ == '__main__':
    main()


FileNotFoundError: ignored